# Simple map with FOLIUM


**Date**: 22.03.2023

**Course**: Methods of Spatial Analysis. Advanced Level. // HSE, Moscow, spring 2023

## 1. Import libraries

In [2]:
## For working with dataframes

import pandas as pd
import geopandas as gpd
import folium

# For working with requests
import requests
import json

In [ ]:
import folium